In [1]:
import pandas as pd
import dataframe_image as dfi

In [2]:
from gcd_table import build_gcd_table_filled, captured_tables

In [3]:
# example from:
#  https://win-vector.com/2024/10/06/dudeneys-remainder-problem/
soln = build_gcd_table_filled(
    508811 - 480608, 
    723217 - 480608,
    verbose=True)

'build row 0'

,a,b,a%b,a//b
0,242609,28203,16985,8
1,,,,
2,,,,
3,,,,
4,,,,
5,,,,
6,,,,
7,,,,


'build row 1'

,a,b,a%b,a//b
0,242609,28203,16985,8
1,28203,16985,11218,1
2,,,,
3,,,,
4,,,,
5,,,,
6,,,,
7,,,,


'build row 2'

...

,a,b,a%b,a//b,"GCD(a, b)",u,v
0,242609,28203,16985,8,79,,
1,28203,16985,11218,1,79,,
2,16985,11218,5767,1,79,-35,53
3,11218,5767,5451,1,79,18,-35
4,5767,5451,316,1,79,-17,18
5,5451,316,79,17,79,1,-17
6,316,79,0,4,79,0,1
7,79,0,N/A,N/A,79,1,0


'back fill row 1'

,a,b,a%b,a//b,"GCD(a, b)",u,v
0,242609,28203,16985,8,79,,
1,28203,16985,11218,1,79,53,-88
2,16985,11218,5767,1,79,-35,53
3,11218,5767,5451,1,79,18,-35
4,5767,5451,316,1,79,-17,18
5,5451,316,79,17,79,1,-17
6,316,79,0,4,79,0,1
7,79,0,N/A,N/A,79,1,0


'back fill row 0'

,a,b,a%b,a//b,"GCD(a, b)",u,v
0,242609,28203,16985,8,79,-88,757
1,28203,16985,11218,1,79,53,-88
2,16985,11218,5767,1,79,-35,53
3,11218,5767,5451,1,79,18,-35
4,5767,5451,316,1,79,-17,18
5,5451,316,79,17,79,1,-17
6,316,79,0,4,79,0,1
7,79,0,N/A,N/A,79,1,0


In [4]:
tables = [t for t in captured_tables]  # copy
captured_tables.clear()

In [5]:
soln['u * a + v * b'] = soln['u'] * soln['a'] + soln['v'] * soln['b']

def highlight_col(x):
    return ['background-color: yellow; font-weight: bold' if col_name == 'u * a + v * b' else '' for col_name in x.index]


tables.append(soln.style.apply(highlight_col, axis=1))

In [6]:
for i, t in enumerate(tables):
    dfi.export(t, f'dataframe_{i:04d}.png', dpi=600)

Then convert at the command line with:

```
ffmpeg -framerate 0.5 -i dataframe_%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p RemainderProblem.mp4
```


In [7]:
!ffmpeg -framerate 0.5 -i dataframe_%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p RemainderProblem.mp4

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1.2_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --